## Grouping similar equations from steps

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import re

In [2]:
data = pd.read_csv("algebra_2008_2009_train_trimmed3_remove_duplicates_sorted.csv", delimiter='\t')

In [3]:
data.describe(include='all')

,Anon Student Id,Problem Name,Problem View,Step Name,First Transaction Time,Correct First Attempt,Incorrects,Hints,Corrects,KC(SubSkills),Opportunity(SubSkills),KC(KTracedSkills),Opportunity(KTracedSkills),KC(Rules),Opportunity(Rules),Unit,Sections
count,8483920,8483920,8.483920e+06,8483920,8483920,8.483920e+06,8.483920e+06,8.483920e+06,8.483920e+06,6204248,6204248,4296385,4296385,8201432,8201432,8483920,8483920
unique,3310,188368,NaN,700635,823557,NaN,NaN,NaN,NaN,1815,910506,921,157173,2780,10780,42,165
top,stu_ea97537407,L5FB16,NaN,FinalAnswer,2009-04-27 12:55:20.0,NaN,NaN,NaN,NaN,Bogus skill,1,Entering a given-1,1,DISTANCE-MIDPOINT-PREFILLED-BOGUS-HINT,1,Unit CTA1_02,Section CTA1_02-2
freq,15608,52755,NaN,255762,109,NaN,NaN,NaN,NaN,408686,229924,440356,243040,408686,436844,715873,527203
mean,NaN,NaN,1.091596e+00,NaN,NaN,8.595012e-01,3.432128e-01,8.589826e-02,1.008225e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,3.611967e-01,NaN,NaN,3.475039e-01,3.904958e+00,4.622137e-01,3.963328e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,1.000000e+00,NaN,NaN,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,1.000000e+00,NaN,NaN,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,1.000000e+00,NaN,NaN,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
steps = data['Step Name'].tolist()

In [53]:
operator_set = set(['+', '=', '/', '*', '^', '>', '<']) # does not include minus sign.
equations = []

for step in steps:
    for char in step:
        if char in operator_set:
            equations.append(step)
            break


In [61]:
operator_set.add('@')
operator_set.add('|')
operator_set.add('&')
operator_set.add(':')
operator_set.add(';')
operator_set.add('`')
operator_set.add('(')
operator_set.add(')')
operator_set.add('#')
operator_set.add("'")
operator_set.add('"')
# parse equations
parsed_equations = []

for index, equation in enumerate(equations):
    to_add = []
    i = 0
    equation = equation.replace('??sqrt', '@')
    equation = equation.replace('sqrt', '@')
    equation = equation.replace(' or ', '|')
    equation = equation.replace(' and ', '&')
    equation = equation.replace('log', '`')
    equation = equation.replace('abs', '#')
    equation = equation.replace('row', "'")
    equation = equation.replace('root', '"')
    while i < len(equation):
            
        if equation[i].isdigit():
            i += 1
            already_decimal = False
            while i < len(equation):
                if equation[i].isdigit():
                    i += 1
                elif equation[i] is ',':
                    i += 1
                elif equation[i] is '.' and not already_decimal:
                    i += 1
                    already_decimal = True
                else:
                    break
        
            to_add.append('~') # ~ represents number
        
        elif equation[i] in operator_set:
            to_add.append(equation[i])
            i += 1
        
        elif equation[i] is '-':
            
            if i == 0 or equation[i-1] in operator_set:
                i += 1
                if equation[i].isdigit():
                    i += 1
                    already_decimal = False
                    while i < len(equation):
                        if equation[i].isdigit():
                            i += 1
                        elif equation[i] is ',':
                            i += 1
                        elif equation[i] is '.' and not already_decimal:
                            i += 1
                            already_decimal = True
                        else:
                            break

                    to_add.append('~') # ~ represents number
                    #to_add.append('-~')  # this would treat all minus signs as an operator
                else:
                    to_add.append('-')
                    
            else:
                i += 1
                to_add.append('-')
        
        elif equation[i].isalpha():
            i += 1
            to_add.append('x') # x is a variable name
        
        else:
            try:
                unicode(equation[i])
                print equation, equation[i], index
                raise AssertionError
                i += 1
            except UnicodeDecodeError:
                i += 1
                pass
    if index > 2000000:
        raise AssertionError
    
    parsed_equations.append("".join(to_add))


In [65]:
zip(equations, parsed_equations)[1100000:1100100]

[('-6x=-8-6', '~x=~-~'),
 ('-6x=-14', '~x=~'),
 ('-6x/-6=-14/-6', '~x/~=~/~'),
 ('x=-14/-6', 'x=~/~'),
 ('-39y+(-28)+y=-40', '~x+(~)+x=~'),
 ('-38y+(-28)=-40', '~x+(~)=~'),
 ('-38y+(-28)-(-28)=-40-(-28)', '~x+(~)-(~)=~-(~)'),
 ('-38y=-40-(-28)', '~x=~-(~)'),
 ('-38y=-12', '~x=~'),
 ('-38y/-38=-12/-38', '~x/~=~/~'),
 ('y=-12/-38', 'x=~/~'),
 ('-33y+(-y)+(-4)=-22', '~x+(-x)+(~)=~'),
 ('-34y+(-4)=-22', '~x+(~)=~'),
 ('-34y+(-4)+4=-22+4', '~x+(~)+~=~+~'),
 ('-34y=-22+4', '~x=~+~'),
 ('-34y=-18', '~x=~'),
 ('-34y/-34=-18/-34', '~x/~=~/~'),
 ('y=-18/-34', 'x=~/~'),
 ('-28=-41x+(-x)+(-36)', '~=~x+(-x)+(~)'),
 ('-28=-42x-36', '~=~x-~'),
 ('-28+36=-42x-36+36', '~+~=~x-~+~'),
 ('8=-42x-36+36', '~=~x-~+~'),
 ('8=-42x', '~=~x'),
 ('8/-42=-42x/-42', '~/~=~x/~'),
 ('-0.19047619=-42x/-42', '~=~x/~'),
 ('8=-6+(-3x)+6x', '~=~+(~x)+~x'),
 ('8=3x-6', '~=~x-~'),
 ('8+6=3x-6+6', '~+~=~x-~+~'),
 ('14=3x-6+6', '~=~x-~+~'),
 ('14=3x', '~=~x'),
 ('14/3=3x/3', '~/~=~x/~'),
 ('4.66666667=3x/3', '~=~x/~'),
 ('-7=

In [66]:
len(set(equations))

697039

In [67]:
len(set(parsed_equations))

17953

In [68]:
def convert_string_if_equation(equation):
    operator_set = set(['+', '=', '/', '*', '^', '>', '<']) # does not include minus sign.
    for char in equation:
        if char in operator_set:
            return parse_equation(equation)
    return equation
        
def parse_equation(equation):
    operator_set = set(['+', '=', '/', '*', '^', '>', '<', '@', '|', '&', 
                       ':', ';', '`', '(', ')', '#', "'", '"'])

    to_add = []
    i = 0
    equation = equation.replace('??sqrt', '@')
    equation = equation.replace('sqrt', '@')
    equation = equation.replace(' or ', '|')
    equation = equation.replace(' and ', '&')
    equation = equation.replace('log', '`')
    equation = equation.replace('abs', '#')
    equation = equation.replace('row', "'")
    equation = equation.replace('root', '"')
    while i < len(equation):

        if equation[i].isdigit():
            i += 1
            already_decimal = False
            while i < len(equation):
                if equation[i].isdigit():
                    i += 1
                elif equation[i] is ',':
                    i += 1
                elif equation[i] is '.' and not already_decimal:
                    i += 1
                    already_decimal = True
                else:
                    break

            to_add.append('~') # ~ represents number

        elif equation[i] in operator_set:
            to_add.append(equation[i])
            i += 1

        elif equation[i] is '-':

            if i == 0 or equation[i-1] in operator_set:
                i += 1
                if equation[i].isdigit():
                    i += 1
                    already_decimal = False
                    while i < len(equation):
                        if equation[i].isdigit():
                            i += 1
                        elif equation[i] is ',':
                            i += 1
                        elif equation[i] is '.' and not already_decimal:
                            i += 1
                            already_decimal = True
                        else:
                            break

                    to_add.append('~') # ~ represents number
                    #to_add.append('-~')  # this would treat all minus signs as an operator
                else:
                    to_add.append('-')

            else:
                i += 1
                to_add.append('-')

        elif equation[i].isalpha():
            i += 1
            to_add.append('x') # x is a variable name

        else:
            try:
                unicode(equation[i])
                print equation, equation[i], index
                raise AssertionError
                i += 1
            except UnicodeDecodeError:
                i += 1
                pass

    return "".join(to_add)


In [69]:
data['Step Name'] = data['Step Name'].apply(convert_string_if_equation)

In [74]:
len(set(data['Step Name'].tolist()))

21549